In [1]:
import numpy as np


## Linear Equations
$$
Ax = b
$$
### Solution 1:
- Fixed point iteration:

    define $G(x) = Ax - b + x = ( A + I)x - b$, and let $G(x)$ be the x in the next iteration
    
    The iterative process converges if all eigenvalues of (A+I) is within the unit circle. However, this condition is too restrict, which is unlikely to happen in real cases. Therefore, we use Gauss-Jacobi, Gauss-Seidel and other iterative shemes instead.
    
    

### Gauss-Jacobi algorithm

In [5]:
def gauss_jacobi(A,b, x):
    # Extract the diagonal entries
    N = np.diagflat(np.diag(A))
    P = N - A
    return np.linalg.solve(N, b + P.dot(x))

In [6]:

A = np.array([[3,1,2,-2],[1,5,2,-3],[2,3,6,3],[1,1,2,-5]])
x = np.zeros(shape = len(A))
b = np.random.uniform(high = 5,size=len(A))
epsilon = 1e-14
pre_x = np.zeros_like(x)
while True:
    pre_x = x
    x = gauss_jacobi(A,b,x)
    if  ((x - pre_x)**2).sum() < epsilon:
        print('Successfully find solutions:', x)
        break

Successfully find solutions: [-1.01708116  0.16642876  0.93157338 -0.72527202]


### Gauss-Seidel algorithm

In [7]:
def gauss_seidel(A,b, x,w = 1,shuffle = False):
        # Allowing dampening and extrapolating with weight w
        # The sequence of solving x's matters for Gauss-Seidel algorithm, we add a shuffle option
    if shuffle == True:
        values = np.concatenate([A,b.reshape(len(b),-1)],axis=1)
        A = values[:,:-1]
        b = values[:,-1]
    
    # Extract the diagonal entries, lower triangular, and upper triangular
    L = np.tril(A)
    U = A - L
        
    return w * np.linalg.solve(L, b - U.dot(x)) + (1 - w) * x

In [12]:
# Wrap the two linear system solves into a Linear Solver
class LinearSolver:
    
    # We allow direct call methods in the class
    @staticmethod
    def gauss_jacobi(A, b, tol=1e-10):
        x = np.zeros_like(b)
        N = np.diagflat(np.diag(A))
        P = A - N
        
        while True:
            pre_x = x.copy()
            x = np.linalg.solve(N, b - P.dot(pre_x))
            if np.sum((x - pre_x) ** 2) < tol:
                print('Successfully found solutions with Jacobi method:', x)
                break
        
        print("Results for Ax - b =", A.dot(x) - b)
        return x
    
    @staticmethod
    def gauss_seidel(A, b, w=1, shuffle=False, tol=1e-10):
        x = np.zeros_like(b)
        
        if shuffle:
            indices = np.arange(len(b))
            np.random.shuffle(indices)
            A = A[indices]
            b = b[indices]
        
        L = np.tril(A)
        U = A - L
        
        while True:
            pre_x = x.copy()
            x = w * np.linalg.solve(L, b - U.dot(pre_x)) + (1 - w) * pre_x
            if np.sum((x - pre_x) ** 2) < tol:
                print('Successfully found solutions with Gauss-Seidel method:', x)
                break
        
        print("Results for Ax - b =", A.dot(x) - b)
        return x


In [13]:
A = np.array([[1,.5,.3],[.6,1,.1],[.2,.4,1]])
b = np.array([5.,7.,4.])


In [14]:
lin_solver = LinearSolver()
lin_solver.gauss_jacobi(A,b)
lin_solver.gauss_seidel(A,b)

Successfully found solutions with Jacobi method: [1.67155639 5.86510467 1.31964982]
Results for Ax - b = [3.67428675e-06 3.48811842e-06 2.97013887e-06]
Successfully found solutions with Gauss-Seidel method: [1.67155553 5.86510185 1.31964815]
Results for Ax - b = [ 9.00075156e-07 -1.41752707e-08  0.00000000e+00]


array([1.67155553, 5.86510185, 1.31964815])

## Optimization

### Polytope
- Polytope method can be used for any $R^n$ functions, regardless of continuity property
- Polytope can guarantee a solution but cannot guarantee a global minimum (maximum)
- If the objective is rough, Polytope method is advised

In [6]:
def polytope_algorithm(f, simplex, epsilon):
    """
    Description:
    Perform the Polytope Algorithm.
    
    Parameters:
    f (function): The objective function to minimize.
    simplex (numpy array): Initial simplex, an (n+1)x(n) array.
    epsilon (float): Stopping rule parameter.
    
    Returns:
    numpy array: The optimized vertex of the simplex.
    """
    
    n = simplex.shape[0]
    
    while True:
        # Step 1: Reorder the simplex vertices so that f(x^1) >= f(x^2) >= ... >= f(x^{n+1})
        simplex = np.array(sorted(simplex, key=lambda x: f(x), reverse=True))
        # Step 2: Find the least i such that f(x^i) > f(y^i) where y^i is the reflection of x^i
        for i in range(n):
            # We shrink the distance of the reflection
            reflected = 1e-3 * np.mean(np.delete(initial_simplex,i,0)) + simplex[i]
            # The first element that exceeds the function value of the reflected
            if f(simplex[i]) > f(reflected):
                simplex[i] = reflected
                break
        # Step 3: Stopping rule, when the simplex is small enough
        if np.sum(abs(simplex[-1] - simplex[0])) < epsilon:
            return simplex[-1]  # Return the best vertex
        # Step 4: Shrink simplex
        simplex = .5 * simplex[-1] + 0.5 * simplex

# Example usage:
def objective_function(x):
    return np.sum((x - np.array([1.3, 0.8,.3]))**2)  # Example: minimize the sum of squares






In [390]:
# For a 3-dimensional problem, simplex (the convex hull) has three vertex
initial_simplex = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])


result = polytope_algorithm(objective_function, initial_simplex, epsilon=1e-14)
print("Optimized vertex:", result)

Optimized vertex: [1.21677517 0.71625434 0.46697049]


### Newton Method
- Iteration scheme: $x^{k + 1} = x^k - H^{-1}(x^k) (\nabla f(x^k))^T$

In [15]:
# For simplicity, I show a 2 dimensional case, and I write a BFGS optimizer more explicitly below 
x = np.array([1.0,5.0])
delta = 1e-10
# Newton's method
while 1:
    print('Current (x,y) = ', x[0], x[1])
    fx = pi(x)
    grads = grad_func(x)
    H = hessian(pi)(x)
    s = np.linalg.solve(H, -grads.reshape(-1,1)).flatten()
    
    if (s**2).sum() < epsilon * (1 + (x**2).sum()):
        print('Iteration ends')
        if (grads**2).sum() > delta  * (1 + np.abs(fx)):
            print('Not optimum')
        else:
            print('reach an optimum')
        print('Success')
        break
    x = x + s

Optimal solution: [10. 14.]


### Quasi-Newton method
#### BFGS

In [4]:
import autograd.numpy as np
from autograd import grad, hessian

class BFGS:
    """
    Description: This an example of a BFGS optimizer. 
    For faster convergence, I comment out the delta criterion
    """
    def __init__(self, func, x, epsilon=1e-12,delta = 1e-10):
        self.func = func
        self.x = np.array(x, dtype=float)
        self.epsilon = epsilon
        self.delta = delta
        self.H = np.eye(len(x))
        self.grad_func = grad(func)
        self.hessian_func = hessian(func)
        
    def line_search(self, s, c1=0.6, c2=0.8):
        a = 0.5
        fx = self.func(self.x)
        grads = self.grad_func(self.x)
        
        while True:
            new_x = self.x + a * s
            new_grads = self.grad_func(new_x)
            new_fx = self.func(new_x)
            
            if (new_fx <= fx + c1 * a * (grads.dot(s)) and 
                np.abs(new_grads.dot(s)) < np.abs(c2 * (grads.dot(s)))):
                return a, new_x, new_grads,new_fx
            a *= 2
            if a > 4:
                return 4, new_x, new_grads,new_fx

    def update_hessian(self, z, y):
        H = self.H
        term1 = H - (H.dot(z.reshape(-1, 1)).dot(z.reshape(1, -1)).dot(H)) / (z.dot(H).dot(z))
        term2 = (y.reshape(-1, 1).dot(y.reshape(1, -1))) / (y.dot(z))
        return term1 + term2

    def optimize(self):
        while True:
            fx = self.func(self.x)
            grads = self.grad_func(self.x)
            H = self.hessian_func(self.x)
            s = np.linalg.solve(H, -grads.reshape(-1, 1)).flatten()
            
            lda, x_new, new_grads,new_fx = self.line_search(s)
            z = lda * s
            y = new_grads - grads
            print('Current x: ',self.x)
            self.x = x_new  # Update x to the new position calculated by line_search
            self.H = self.update_hessian(z, y)

            print('Current Function Value= ', fx)
            if (lda < self.delta) or ((z**2).sum() < self.epsilon * (1 + (self.x**2).sum())):
                print('Iteration ends')
#                 if (grads**2).sum() > self.delta * (1 + np.abs(fx)):
#                     print('Not optimum')
#                 else:
#                     print('Reached an optimum')
                print('Success')
                break



In [8]:
# Simple Example
def fun(x):
    x0, x1,x2 = x
    u = x0**(.3) * x1**(.2)*x2**(.5)
    return (u-10)**2

bfgs = BFGS(func= fun, x=np.random.uniform(size=3))
bfgs.optimize()

## You can find a more useful example in the script ``logit_example'' in repo ``DiscreteChoiceModel_2024Summer'', 
## where I use BFGS optimizer to estimate the plain logit model using maximum likelihood estimator. 

Current x:  [0.72993383 0.28998015 0.10239402]
Current Function Value=  95.50570054925159
Current x:  [16.42171173  6.5238385   2.30361295]
Current Function Value=  23.87642513731291
Current x:  [24.26760068  9.64076767  3.40422242]
Current Function Value=  5.969106284328232
Current x:  [28.19054516 11.19923226  3.95452715]
Current Function Value=  1.492276571082058
Current x:  [30.1520174  11.97846455  4.22967952]
Current Function Value=  0.3730691427705145
Current x:  [31.13275351 12.3680807   4.3672557 ]
Current Function Value=  0.09326728569262754
Current x:  [31.62312157 12.56288877  4.43604379]
Current Function Value=  0.023316821423157157
Current x:  [31.8683056  12.66029281  4.47043784]
Current Function Value=  0.005829205355788747
Current x:  [31.99089762 12.70899483  4.48763486]
Current Function Value=  0.0014573013389473902
Current x:  [32.05219363 12.73334584  4.49623337]
Current Function Value=  0.0003643253347367797
Current x:  [32.08284163 12.74552134  4.50053263]
Curren

In [9]:
# Also, we can use BFGS to solve the polytope example
bfgs = BFGS(func= objective_function, x=np.random.uniform(size=3))
bfgs.optimize()


Current x:  [0.1619784  0.56297325 0.02029818]
Current Function Value=  1.429507938600607
Current x:  [0.7309892  0.68148663 0.16014909]
Current Function Value=  0.3573769846501518
Current x:  [1.0154946  0.74074331 0.23007454]
Current Function Value=  0.089344246162538
Current x:  [1.1577473  0.77037166 0.26503727]
Current Function Value=  0.02233606154063453
Current x:  [1.22887365 0.78518583 0.28251864]
Current Function Value=  0.0055840153851586145
Current x:  [1.26443683 0.79259291 0.29125932]
Current Function Value=  0.0013960038462896458
Current x:  [1.28221841 0.79629646 0.29562966]
Current Function Value=  0.0003490009615724071
Current x:  [1.29110921 0.79814823 0.29781483]
Current Function Value=  8.72502403930998e-05
Current x:  [1.2955546  0.79907411 0.29890741]
Current Function Value=  2.1812560098273964e-05
Current x:  [1.2977773  0.79953706 0.29945371]
Current Function Value=  5.453140024567997e-06
Current x:  [1.29888865 0.79976853 0.29972685]
Current Function Value=  1

## Nonlinear Equations
- General forms for a non-linear system: $f:R^n \rightarrow R^n$

zeros of f: $f(x) = 0$

fixed points of f: $ f(x) = x$

### One-dimension
#### Bisection (linear convergence)

In [115]:
# Without losing generality, we assume a zero point exists between (0,10]
epsilon = 1e-10
def bisect(fun,a = 0.1):
    init_fa = fun(a)
    for potential_b in np.arange(a,4,0.1):
        if fun(potential_b) * init_fa < 1e-10:
            b = potential_b
            break
    else:
        print('No potential b\'s satisfy initialization criterion, choose another b range...')
        return -1
        
    while True:
        interm = (a+b)/2
        fa = fun(a)
        fb = fun(b)
        if fa * fun(interm) <1e-10:
            b = interm
        else:
            a = interm
        print(interm)
        if abs(b - a) < epsilon:
            print('zero point has been found: ', interm)
            break
        

In [77]:
# Example: we consider demand function, D = 2/p + 3; while supply function, S = 4p**3 - 2.
# Therefore, the excess demand function, ED = 2/p + 3 - (4 * exp(p) - 2)

def ed(p):
    return 2/p + 3 - (4*np.exp(p) - 2)

In [78]:
bisect(ed)

0.55
0.775
0.6625000000000001
0.71875
0.690625
0.6765625000000001
0.68359375
0.687109375
0.6853515625
0.68447265625
0.684033203125
0.6838134765625
0.68370361328125
0.683758544921875
0.6837310791015625
0.6837448120117188
0.6837379455566406
0.6837413787841797
0.6837396621704102
0.6837388038635254
0.6837392330169678
0.683739447593689
0.6837393403053285
0.6837393939495087
0.6837394207715988
0.6837394341826439
0.6837394274771214
0.6837394308298826
0.6837394325062633
0.6837394316680729
0.6837394312489777
0.6837394310394302
0.6837394309346565
0.6837394309870434
zero point has been found:  0.6837394309870434


#### Newton's method (one-dimension case)
- Linear approximation: $g(x) = f'(x_k)(x - x_k) + f(x_k)$
- Iteration scheme: $x_{k + 1} = x_k - \frac{f(x_k)}{f'(x_k)}$
- With a hope that the sequence $x_k$ will converge to the zero of $f$ (quadratically convergent)

In [85]:
def newton(fun,x0 = 0.1):
    epsilon = 1e-10
    delta = 1e-10
    x = x0
    grad_fun = grad(fun)
    while True:
        s = fun(x)/ grad_fun(x)
        x -= s
        print(x)
        if abs(s)<epsilon * (1 + abs(x)):
            print('Iteration ends......')
            if abs(fun(x)) < delta:
                print('Solution has been found: ', x)
            else:
                print('No solution has been found')
            break

In [86]:
newton(ed)

0.2006713995766063
0.385396411392974
0.6081124116895442
0.6818824743584165
0.6837387720542631
0.6837394309346968
0.6837394309347784
Iteration ends......
Solution has been found:  0.6837394309347784


In [182]:
lb = 1e-5
p1 = 0
fun(lb) + grad_fun(lb) * (p1 - lb) + (p1 - lb)**2 * grad(grad_fun)(lb)/2

-8.401580009820094e-05

In [165]:
# A simple CGE exmple with two consumers and two products
# Following Walras' law, we only need to consider (n-1) market and we normalize p1 + p2 = 1
r = -5
a12 = 1
a21 = 1
a11 = 1024
a22 = 1024
e11 = 12
e22 = 12
e21 = 1
e12 = 1

def fun(p1):
    p2 = 1 - p1
    x12 = (p1 * e11 + p2 * e12)/(p1 * (p1 * a12/p2/a11)**(1/r) + p2)
    x22 = (p1 * e21 + p2 * e22)/(p1 * (p1 * a22/p2/a21)**(1/r) + p2)
    return x12 + x22 - e12 - e22



# Since price has to be non-negative,a.k.a., limited domain, we replace original function with fun_tilde
grad_fun = grad(fun)
def fun_tilde(p1):
    lb = 1e-5
    ub = 1 - 1e-5
    if p1 < lb:
        return fun(lb) + grad_fun(lb) * (p1 - lb) + (p1 - lb)**2 * grad(grad_fun)(lb)/2
    elif p1 > ub:
        return fun(ub) + grad_fun(ub) * (p1 - ub) + (p1 - ub)**2 * grad(grad_fun)(ub)/2
    else:
        return fun(p1)


In [117]:
# This CGE has three equilibria, we can get all of them by setting different initial values
bisect(fun,a = .3)

0.4
0.45
0.475
0.4875
0.49375
0.496875
0.4984375
0.49921875
0.499609375
0.4998046875
0.49990234375
0.499951171875
0.4999755859375
0.49998779296875
0.49998168945312504
0.4999847412109375
0.49998321533203127
0.49998245239257816
0.49998207092285163
0.49998188018798834
0.49998178482055666
0.49998173713684085
0.499981713294983
0.499981701374054
0.4999816954135895
0.49998169243335727
0.4999816909432412
0.4999816901981831
0.49998168982565405
0.49998168963938955
0.4999816895462573
zero point has been found:  0.4999816895462573


In [118]:
newton(fun)

0.11254257722463024
0.1129234593732251
0.11292384712759874
0.11292384712800163
Iteration ends......
Solution has been found:  0.11292384712800163


In [121]:
newton(fun,x0=.9)

0.8889915781252126
0.8871218805566008
0.8870761793372882
0.887076152872008
0.8870761528719988
Iteration ends......
Solution has been found:  0.8870761528719988


In [131]:
# Suffer limited domain issue
newton(fun,x0=0.01)

-0.021535345443671083
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
na

/var/folders/vq/y8sw0dx95kg8d24dbz5s_kt80000gn/T/ipykernel_55479/2275684538.py:15: RuntimeWarning: invalid value encountered in scalar power
  x12 = (p1 * e11 + p2 * e12)/(p1 * (p1 * a12/p2/a11)**(1/r) + p2)
/var/folders/vq/y8sw0dx95kg8d24dbz5s_kt80000gn/T/ipykernel_55479/2275684538.py:16: RuntimeWarning: invalid value encountered in scalar power
  x22 = (p1 * e21 + p2 * e22)/(p1 * (p1 * a22/p2/a21)**(1/r) + p2)



nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


KeyboardInterrupt: 

In [183]:
# Avoided the limited domain issue, but point x=0 is not a solution
newton(fun_tilde,x0=0.01)

-0.021535345443671083
-0.010743556341296746
-0.005347719412701803
-0.0026499161892814507
-0.001301245024871699
-0.0006273700617935158
-0.00029135161970622077
-0.00012516304547408863
-4.557548683428039e-05
-1.1855598452631391e-05
-2.4123860850689766e-06
-1.534001841631761e-06
-1.526268002423859e-06
-1.5262674027934035e-06
Iteration ends......
Solution has been found:  -1.5262674027934035e-06


### Multivariate nonlinear equation system
- Suppose $f: R^n \rightarrow R^n$ and we want to solve $f(x) = 0$, where 
\begin{align}
    f(x) =0:~ & \begin{bmatrix}
           f^1(x_1,x_2,\dots,x_n)=0 \\
           f^2(x_1,x_2,\dots,x_n)=0 \\
           \vdots \\
            f^n(x_1,x_2,\dots,x_n)=0 \\
         \end{bmatrix}
  \end{align}

#### Gauss-Jacobi Algorithm
- Similar to Gauss-Jacobi algorithm in linear system, update x in the (k+1)-th iteration using one equation
\begin{bmatrix}
           f^1(x^{k+1}_1,x^k_2,\dots,x^k_n)=0 \\
           f^2(x^k_1,x^{k+1}_2,\dots,x^k_n)=0 \\
           \vdots \\
            f^n(x^k_1,x^k_2,\dots,x^{k+1}_n)=0 \\
         \end{bmatrix}

- For each equation, we apply one-dimensional Newton's method
$$
x^{k+1}_i = x^k_i - \frac{f^i(x^k)}{f^i_{x_i}(x_k)}
$$

In [184]:
from autograd import jacobian

In [207]:

def fun(p):
    p1,p2 = p
    x12 = (p1 * e11 + p2 * e12)/(p1 * (p1 * a12/p2/a11)**(1/r) + p2)
    x22 = (p1 * e21 + p2 * e22)/(p1 * (p1 * a22/p2/a21)**(1/r) + p2)
    return np.array([x12 + x22 - e12 - e22,x12 + x22 - e12 - e22])



In [213]:
inputs = np.array([0.5,0.5])
jacobian(fun)

<function autograd.wrap_util.unary_to_nary.<locals>.nary_operator.<locals>.nary_f(*args, **kwargs)>

In [285]:
def fun(params):
    x,y = params
    return np.array([y-np.exp(x), x * (x**2 - 3*y)**2])

In [286]:
jac_inv = np.zeros(2)
inputs = np.array([0.5,0.5])
jac_func = jacobian(fun)
jac_inv = 1/np.diag(jac_func(inputs))

In [287]:
jac_func(np.array([0.5,0.5]))

array([[-1.64872127,  1.        ],
       [ 0.3125    ,  3.75      ]])

In [288]:
def gauss_jacobi(fun, x0 = np.array([0.1,0.1])):
    epsilon = 1e-10
    delta = 1e-10
    jac_func = jacobian(fun)
    x = x0
    while True:
        fx = fun(x)
        jac_inv = 1/np.diag(jac_func(x))
        s = fx * jac_inv
        x -= s
        print(x)
        if (s**2).sum() < epsilon * (1 + (s**2).sum()):
            print('Iteration ends......')
            if (fun(x)**2).sum() < delta:
                print('Solution has been found: ', x)
            else:
                print('No solution has been found')
            break

In [297]:
gauss_jacobi(fun,x0 = np.array([0.1,0.1]))

[-0.80951626  0.05166667]
[-1.69343051  0.13505276]
[-1.95899952  0.54547753]
[0.90965144 0.91235195]
[0.27702323 0.59408693]
[-0.27263696  0.30983378]
[-0.86569405  0.16730537]
[-1.46806689  0.20855705]
[-1.56275499  0.46348193]
[-0.35104249  0.63877482]
[-0.44363239  0.33992588]
[-0.91390634  0.20276456]
[-1.40820218  0.24058641]
[-1.424541    0.45079877]
[-0.55104394  0.5636189 ]
[-0.57312944  0.33241768]
[-0.98348382  0.22095507]
[-1.39270412  0.27168427]
[-1.29897894  0.45911293]
[-0.61607661  0.51078085]
[-0.67028975  0.31864882]
[-1.04739387  0.2342058 ]
[-1.37985735  0.29994189]
[-1.18778791  0.46730533]
[-0.65511151  0.46879268]
[-0.75251814  0.30592486]
[-1.1032423   0.24734302]
[-1.35776967  0.32652877]
[-1.08838476  0.4705208 ]
[-0.69118558  0.43269063]
[-0.82750019  0.29596823]
[-1.15044532  0.26211021]
[-1.322282    0.35164251]
[-1.00292858  0.4672262 ]
[-0.72915119  0.40125739]
[-0.89721622  0.28923894]
[-1.18778089  0.27878562]
[-1.27342122  0.37453005]
[-0.93520441  0.

[-1.0327327   0.35580618]
[-1.03336893  0.35565923]
[-1.03378214  0.35580484]
[-1.03337282  0.35602001]
[-1.03276817  0.35598657]
[-1.03286228  0.35576163]
[-1.03349415  0.35568156]
[-1.03371941  0.35585914]
[-1.03322018  0.35602554]
[-1.0327527   0.35593676]
[-1.03300218  0.3557314 ]
[-1.03357915  0.35571462]
[-1.0336265   0.35590495]
[-1.03309143  0.35601643]
[-1.03277834  0.35588787]
[-1.03313952  0.35571578]
[-1.03362312  0.3557541 ]
[-1.03351551  0.35593984]
[-1.03299341  0.35599564]
[-1.03283679  0.35584372]
[-1.03326355  0.35571383]
[-1.03362865  0.35579584]
[-1.03339816  0.35596262]
[-1.03292944  0.3559666 ]
[-1.03291836  0.35580717]
[-1.03336624  0.35572364]
[-1.0336011   0.35583612]
[-1.03328494  0.35597327]
[-1.03289957  0.35593293]
[-1.03301296  0.35578005]
[-1.03344248  0.35574266]
[-1.03354768  0.35587189]
[-1.03318441  0.35597274]
[-1.03290107  0.35589804]
[-1.03311096  0.35576313]
[-1.03349007  0.35576794]
[-1.03347662  0.35590092]
[-1.03310282  0.35596278]
[-1.03292908

#### Gauss-Seidel algorithm
- We seperately define each function, which leads to faster calculation speed

In [298]:
def fun1(params):
    x,y = params
    return y-np.exp(x)

def fun2(params):
    x,y = params
    return x * (x**2 - 3*y)**2

In [303]:
import autograd.numpy as np

In [366]:

def gauss_seidel(funs, x0 = np.array([0.1,0.1])):
    fun1, fun2 = funs
    epsilon = 1e-10
    delta = 1e-10
    x = np.array(x0, dtype=float)
    grad_func1 = grad(fun1)
    grad_func2 = grad(fun2)
    s = np.zeros_like(x)
    past_x = []
    while True:
        print(x)
        past_x.append(x.copy())
        f1x = fun1(x)
        grad1 = grad_func1(x)
        s[0] = f1x/grad1[0]
        x[0] -= s[0]
        grad2 = grad_func2(x)
        f2x = fun2(x)
        s[1] = f2x/grad2[1]
        x[1] -= s[1]
        # estimate rho_hat, the spectral radius of the Jacobian matrix
        # 4-degree lag
        if len(past_x)>5:
            # to aviod accuracy loss, we additionally subtract 1e-5
            b = max((((np.array(demo[-3:])/np.array(demo[-4:-1]) - 1e-5)**2)**2).sum(axis=1))
            if (s**2).sum() < epsilon * b:
                print('Iteration ends......')
                if (fun1(x)**2 + fun2(x)**2) < delta:
                    print('Solution has been found: ', x)
                else:
                    print('No solution has been found')
                break

# Find the zero
gauss_seidel([fun1,fun2])


[0.1 0.1]
[-0.80951626  0.15921943]
[-1.45177873  0.43088663]
[-0.61158941  0.27778358]
[-1.09953457  0.34038783]
[-1.07742883  0.3636694 ]
[-1.00928889  0.35161204]
[-1.04458876  0.35766697]
[-1.02801717  0.35497004]
[-1.0356923   0.35626144]
[-1.03208394  0.3556636 ]
[-1.03376853  0.35594469]
[-1.03297961  0.35581349]
[-1.03334852  0.35587494]
[-1.03317589  0.35584621]
[-1.03325664  0.35585965]
[-1.03321886  0.35585336]
[-1.03323654  0.3558563 ]
Iteration ends......
Solution has been found:  [-1.03322827  0.35585493]
